# Pre-condition: Setup Environment, download and install Lib

In [ ]:
# !.venv/bin/activate 
%pip install selenium
%pip install Pillow
%pip install webdriver-manager


# Ý tưởng thực hiện

<img src="image.png" alt="drawing" width="65%"/>


- Xây dựng con bot chơi game, cụ thể ở đây chọn game Dino (chrome://dino/).
- Lí do:
  - Game có cách thức chơi rất đơn giản -> dễ tạo bot hơn.
  - Tạo được bot game đơn giản trước rồi mở rộng ý tưởng cho các game phức tạp hơn cũng dễ.

# Ý tưởng Auto-play

- Tốc độ của máy móc nhanh con người ->
  - Check vùng trước mặt của dino có chướng ngại vật, nếu có thì action để né.
- Sự mạnh mẽ và bền bĩ của máy móc hơn con người ->
  - Loop điều kiện trên liên tục để chẹck trong khoảng thời gian 1s, 0.5s, 0,1s... một cách nhanh nhất tuỳ vào CPU, GPU của máy.
- Máy móc chưa thông minh hơn con người ?

# Tìm hiểu game, cheat.

- Bất tử: ```Runner.prototype.gameOver = function(){}```
- Speed slow: ```Runner.instance_.setSpeed(1)```
- Nhảy cao: ```Runner.instance_.tRex.setJumpVelocity(10)```
- Cần đẩu vân: ```Runner.instance_.tRex.groundYPos = 0```
  

# Code phần bot Auto-play

##### Import các package cần thiết
- `Selenium`
  - The selenium package is used to automate web browser interaction from Python.
  - Several browsers/drivers are supported (Firefox, Chrome, Internet Explorer), as well as the Remote protocol.
  - https://www.selenium.dev/documentation/

In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options


##### Nhận diện vật thể

- chướng ngại vật: `Runner.instance_.horizon.obstacles[0]`
- Dino: `Runner.instance_.tRex`
- 

##### Tính toán khoảng cách từ dino tới chướng ngại vật

In [ ]:
def get_distance_obstacles(self):
        fist_of_dino = self._driver.execute_script(
            "return Runner.instance_.tRex.config.WIDTH_DUCK"
        )
        x_pos_obstacles = self._driver.execute_script(
            """
                const obstacle = Runner.instance_.horizon.obstacles[0]
                if(obstacle){
                    return obstacle.xPos
                }
            """
        )

        if type(fist_of_dino) is int and type(x_pos_obstacles) is int:
            return x_pos_obstacles - fist_of_dino
        return 0

##### Loop tính toán

In [ ]:
from SeleniumHelper import GameDino
import time

dinoPlayer = GameDino()
dinoPlayer.press_up()

while True:
    time.sleep(0.01)
    if dinoPlayer.is_obstacle_nearby():
        # dinoPlayer.press_up()
        dinoPlayer.wrap_press_up()

In [ ]:
def wrap_press_up(self):
    self._driver.execute_script(
        """
    const KEY_CODE_SPACE_BAR = 32
    const KEY_CODE_ARROW_DOWN = 40
    const CANVAS_HEIGHT = Runner.instance_.dimensions.HEIGHT
    const DINO_HEIGHT = Runner.instance_.tRex.config.HEIGHT

    const obstacle = Runner.instance_.horizon.obstacles[0]
    const speed = Runner.instance_.currentSpeed

    if (obstacle) {
        const w = obstacle.width
        const x = obstacle.xPos // measured from left of canvas
        const y = obstacle.yPos // measured from top of canvas
        const yFromBottom = CANVAS_HEIGHT - y - obstacle.typeConfig.height
        
        if (yFromBottom > DINO_HEIGHT) {
            // Pterodactyl going from above, do nothing
        } else if (y > CANVAS_HEIGHT / 2) {
            // cactus
            document.dispatchEvent(new KeyboardEvent("keyup", {keyCode: KEY_CODE_ARROW_DOWN}))
            document.dispatchEvent(new KeyboardEvent("keydown", {keyCode: KEY_CODE_SPACE_BAR}))
        } else {
            // Duck
            document.dispatchEvent(new KeyboardEvent("keydown", {keyCode: KEY_CODE_ARROW_DOWN}))
        }
    }
        """
    )